## Deep + Shallow Water

In [ ]:
import numpy as np
import warnings
warnings.simplefilter("ignore")

from rasterio.transform import Affine
from rasterio.features import geometry_mask


def get_clip_mask(geometries, transform : Affine, shape = tuple) -> np.ndarray:
    return geometry_mask(
        geometries = geometries,
        out_shape = shape,
        transform = transform,
        invert = True
    )

In [ ]:
import glob
import xarray as xr
import numpy as np
import os
import datetime
import pandas as pd
import rasterio
import uav
import geopandas as gpd

from tqdm import tqdm


ndwi_filename = r'D:\sentinel2-acolite-prod\jupyters\Mar\masks\ndwi.tif'

with rasterio.open(ndwi_filename) as src:
    is_water = src.read() > 0.1
    water_mask_tif = uav.GeoreferenceData(is_water, src.transform)

with rasterio.open(r'D:\sentinel2-acolite-prod\jupyters\Mar\base_georeference_reduced.tif') as reference:
    transform = reference.transform
    height, width = reference.read(1).shape


folders = ['section_1', 'section_2', 'section_3', 'whole']
names = ['North', 'Central', 'South', 'Total']

variables = ['SPM_Nechad2016_665', 'SPM_Nechad2016_704', 'SPM_Nechad2016_740', 'TUR_Nechad2016_665', 
             'TUR_Nechad2016_704', 'TUR_Nechad2016_740', 'chl_oc2', 'chl_oc3', 'chl_re_gons', 'chl_re_gons740', 
             'chl_re_mishra', 'chl_re_moses3b', 'chl_re_moses3b740', 'ndci', 'ndvi', 'fai', 'fait', 
             'qaa_v6_KPAR_Lee', 'qaa_v6_Kd_490', 'qaa_v6_Zeu_Lee', 'p3qaa_KPAR', 'p3qaa_zSD']

out_folder = r'D:\sentinel2-acolite-prod\jupyters\Mar\results\shallow_and_deep_water\csv\series_temporales'
os.makedirs(out_folder, exist_ok = True)

for variable in variables:
    print(variable)
    
    data = {name: {'mean': [], 'median': [], 'std': []} for name in names}
    dates = []
    
    for filename in tqdm(glob.glob(rf'D:\NRT-Sentinel2\outputs\FRP_Lagoon\section_1\FRP\*\*.nc')):
        date = os.path.basename(os.path.dirname(filename)).split('_')[0]
        dates.append(datetime.datetime.strptime(date, '%Y%m%d'))
        
        with xr.open_dataset(filename) as src:
            xs, ys = np.meshgrid(src.x, src.y)
            rows, cols = water_mask_tif.get_row_col_by_lon_lat(xs, ys)
            in_shape = water_mask_tif.get_in_shape_mask(rows, cols)
            is_water_mask = water_mask_tif.get_data_by_row_col(rows[in_shape], cols[in_shape])[0].reshape((height, width))
            
            if variable in src:
                values = src[variable].values
                values[np.isinf(values)] = np.nan

                for name, folder in zip(names, folders):
                    shape = gpd.read_file(rf'D:\NRT-Sentinel2\resources\coastlines\Lagoa\{folder}.shp').geometry
                    section_mask = get_clip_mask(shape, transform, (height, width))
                    
                    masked_values = values.copy()
                    masked_values[~section_mask] = np.nan
                    masked_values[~is_water_mask] = np.nan
                    
                    data[name]['mean'].append(np.nanmean(masked_values))
                    data[name]['median'].append(np.nanmedian(masked_values))
                    data[name]['std'].append(np.nanstd(masked_values))
            else:
                for name in names:
                    data[name]['mean'].append(np.nan)
                    data[name]['median'].append(np.nan)
                    data[name]['std'].append(np.nan)

    df_data = {f'{name} {stat}': data[name][stat] for name in names for stat in ['mean', 'median', 'std']}
    pd.DataFrame(df_data, index=dates).to_csv(rf'{out_folder}\{variable}.csv')

## Deep Water

In [ ]:
import glob
import xarray as xr
import numpy as np
import os
import datetime
import pandas as pd
import rasterio
import uav
import geopandas as gpd

from tqdm import tqdm


ndwi_filename = r'D:\sentinel2-acolite-prod\jupyters\Mar\masks\ndwi.tif'

with rasterio.open(ndwi_filename) as src:
    is_water = src.read() > 0.1
    water_mask_tif = uav.GeoreferenceData(is_water, src.transform)

with rasterio.open(r'D:\sentinel2-acolite-prod\jupyters\Mar\base_georeference_reduced.tif') as reference:
    transform = reference.transform
    height, width = reference.read(1).shape


folders = ['section_1', 'section_2', 'section_3', 'whole']
names = ['North', 'Central', 'South', 'Total']

variables = ['SPM_Nechad2016_665', 'SPM_Nechad2016_704', 'SPM_Nechad2016_740', 'TUR_Nechad2016_665', 
             'TUR_Nechad2016_704', 'TUR_Nechad2016_740', 'chl_oc2', 'chl_oc3', 'chl_re_gons', 'chl_re_gons740', 
             'chl_re_mishra', 'chl_re_moses3b', 'chl_re_moses3b740', 'ndci', 'ndvi', 'fai', 'fait', 
             'qaa_v6_KPAR_Lee', 'qaa_v6_Kd_490', 'qaa_v6_Zeu_Lee', 'p3qaa_KPAR', 'p3qaa_zSD']

out_folder = r'D:\sentinel2-acolite-prod\jupyters\Mar\results\deep_water\csv\series_temporales'
os.makedirs(out_folder, exist_ok = True)

with xr.open_dataset(r'D:\sentinel2-acolite-prod\jupyters\Mar\masks\deep_water.nc') as deep_src:
    for variable in variables:
        print(variable)
        
        data = {name: {'mean': [], 'median': [], 'std': []} for name in names}
        dates = []
        
        for filename in tqdm(glob.glob(rf'D:\NRT-Sentinel2\outputs\FRP_Lagoon\section_1\FRP\*\*.nc')):
            date = os.path.basename(os.path.dirname(filename)).split('_')[0]
            dates.append(datetime.datetime.strptime(date, '%Y%m%d'))
            year = date[:4]

            deep_water_mask = deep_src[f'deep water mask {year}'].values
            
            with xr.open_dataset(filename) as src:
                xs, ys = np.meshgrid(src.x, src.y)
                rows, cols = water_mask_tif.get_row_col_by_lon_lat(xs, ys)
                in_shape = water_mask_tif.get_in_shape_mask(rows, cols)
                is_water_mask = water_mask_tif.get_data_by_row_col(rows[in_shape], cols[in_shape])[0].reshape((height, width))
                
                if variable in src:
                    values = src[variable].values
                    values[np.isinf(values)] = np.nan

                    for name, folder in zip(names, folders):
                        shape = gpd.read_file(rf'D:\NRT-Sentinel2\resources\coastlines\Lagoa\{folder}.shp').geometry
                        section_mask = get_clip_mask(shape, transform, (height, width))
                        
                        masked_values = values.copy()
                        masked_values[~section_mask] = np.nan
                        masked_values[~is_water_mask] = np.nan
                        masked_values[~deep_water_mask] = np.nan
                        
                        data[name]['mean'].append(np.nanmean(masked_values))
                        data[name]['median'].append(np.nanmedian(masked_values))
                        data[name]['std'].append(np.nanstd(masked_values))
                else:
                    for name in names:
                        data[name]['mean'].append(np.nan)
                        data[name]['median'].append(np.nan)
                        data[name]['std'].append(np.nan)

        df_data = {f'{name} {stat}': data[name][stat] for name in names for stat in ['mean', 'median', 'std']}
        pd.DataFrame(df_data, index=dates).to_csv(rf'{out_folder}\{variable}.csv')

## Shallow Water

In [ ]:
import glob
import xarray as xr
import numpy as np
import os
import datetime
import pandas as pd
import rasterio
import uav
import geopandas as gpd

from tqdm import tqdm


ndwi_filename = r'D:\sentinel2-acolite-prod\jupyters\Mar\masks\ndwi.tif'

with rasterio.open(ndwi_filename) as src:
    is_water = src.read() > 0.1
    water_mask_tif = uav.GeoreferenceData(is_water, src.transform)

with rasterio.open(r'D:\sentinel2-acolite-prod\jupyters\Mar\base_georeference_reduced.tif') as reference:
    transform = reference.transform
    height, width = reference.read(1).shape


folders = ['section_1', 'section_2', 'section_3', 'whole']
names = ['North', 'Central', 'South', 'Total']

variables = ['SPM_Nechad2016_665', 'SPM_Nechad2016_704', 'SPM_Nechad2016_740', 'TUR_Nechad2016_665', 
             'TUR_Nechad2016_704', 'TUR_Nechad2016_740', 'chl_oc2', 'chl_oc3', 'chl_re_gons', 'chl_re_gons740', 
             'chl_re_mishra', 'chl_re_moses3b', 'chl_re_moses3b740', 'ndci', 'ndvi', 'fai', 'fait', 
             'qaa_v6_KPAR_Lee', 'qaa_v6_Kd_490', 'qaa_v6_Zeu_Lee', 'p3qaa_KPAR', 'p3qaa_zSD']

out_folder = r'D:\sentinel2-acolite-prod\jupyters\Mar\results\shallow_water\csv\series_temporales'
os.makedirs(out_folder, exist_ok = True)

with xr.open_dataset(r'D:\sentinel2-acolite-prod\jupyters\Mar\masks\deep_water.nc') as deep_src:
    for variable in variables:
        print(variable)
        
        data = {name: {'mean': [], 'median': [], 'std': []} for name in names}
        dates = []
        
        for filename in tqdm(glob.glob(rf'D:\NRT-Sentinel2\outputs\FRP_Lagoon\section_1\FRP\*\*.nc')):
            date = os.path.basename(os.path.dirname(filename)).split('_')[0]
            dates.append(datetime.datetime.strptime(date, '%Y%m%d'))
            year = date[:4]

            deep_water_mask = deep_src[f'deep water mask {year}'].values
            
            with xr.open_dataset(filename) as src:
                xs, ys = np.meshgrid(src.x, src.y)
                rows, cols = water_mask_tif.get_row_col_by_lon_lat(xs, ys)
                in_shape = water_mask_tif.get_in_shape_mask(rows, cols)
                is_water_mask = water_mask_tif.get_data_by_row_col(rows[in_shape], cols[in_shape])[0].reshape((height, width))
                
                if variable in src:
                    values = src[variable].values
                    values[np.isinf(values)] = np.nan

                    for name, folder in zip(names, folders):
                        shape = gpd.read_file(rf'D:\NRT-Sentinel2\resources\coastlines\Lagoa\{folder}.shp').geometry
                        section_mask = get_clip_mask(shape, transform, (height, width))
                        
                        masked_values = values.copy()
                        masked_values[~section_mask] = np.nan
                        masked_values[~is_water_mask] = np.nan
                        masked_values[deep_water_mask] = np.nan
                        
                        data[name]['mean'].append(np.nanmean(masked_values))
                        data[name]['median'].append(np.nanmedian(masked_values))
                        data[name]['std'].append(np.nanstd(masked_values))
                else:
                    for name in names:
                        data[name]['mean'].append(np.nan)
                        data[name]['median'].append(np.nan)
                        data[name]['std'].append(np.nan)

        df_data = {f'{name} {stat}': data[name][stat] for name in names for stat in ['mean', 'median', 'std']}
        pd.DataFrame(df_data, index=dates).to_csv(rf'{out_folder}\{variable}.csv')